In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import pandas as pd
import nltk
from nltk.corpus import stopwords
import re
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
import pickle
nltk.download('stopwords')
pkl_filename = "theModel.pkl" # name for saving and loading model
data = pd.read_csv('tweet.csv')
X, y= data['Tweet'], data['class_label']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\2020\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#text preprocessing
documents = []
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [3]:
#Removeing stopword
stw = stopwords.words('english')
addi = ("&amp","arc",'https','co','t','amp','s')
stw.append(addi)
#Do Count vectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1200, min_df=9, max_df=0.8, stop_words=stw)
X = vectorizer.fit_transform(documents).toarray()
y = data['class_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
data.sample(5)

,Unnamed: 0,Id,User,Tweet,class_label
1412,1412,1.340000e+18,Ravi Shankar Prasad,@SonuSood # शिक्षा के लिए स्कूल।\nताकि जागरूकत...,NEGATIVE
606,606,5.572044e+08,SAVE OUR COUNTRY,IS #Trudeau #COVID19 #CovidHotels #QuarentineH...,NEGATIVE
54,54,7.400000e+17,Southern Regional Health Authority,"Please see the #COVID19 update for Thursday, F...",POSITIVE
76,76,7.289820e+07,KCBS Radio - The Traffic Leader,.@KCBSRadio Coming up:\n-#COVID19 relief bill ...,POSITIVE
1184,1184,3.071162e+09,Hong Kong Free Press HKFP,"At Tuesday's weekly press conference, Chief Ex...",POSITIVE


In [4]:
# XGBoost model
#model = XGBClassifier(eta=0.5,gamma=0.14,max_depth=7, random_state=42)
model = XGBClassifier(eta=0.5,gamma=0.35,max_depth=7, random_state=42)
model.fit(X_train, y_train)
# make predictions for test data
predictt = model.predict(X_test)
print(classification_report(y_test ,predictt))
print(confusion_matrix(y_test, predictt))
accuracy = accuracy_score(y_test, predictt)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

C:\Users\2020\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
[16:23:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

    NEGATIVE       0.74      0.84      0.79       183
    POSITIVE       0.69      0.55      0.61       117

    accuracy                           0.73       300
   macro avg       0

In [5]:
#Multinomial
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=0.3)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(classification_report(y_test ,pred ))
print('Confusion Matrix: \n', confusion_matrix(y_test,pred))
print()
print('Accuracy: ', accuracy_score(y_test,pred) * 100.0)

              precision    recall  f1-score   support

    NEGATIVE       0.72      0.83      0.77       183
    POSITIVE       0.65      0.50      0.56       117

    accuracy                           0.70       300
   macro avg       0.69      0.66      0.67       300
weighted avg       0.69      0.70      0.69       300

Confusion Matrix: 
 [[152  31]
 [ 59  58]]

Accuracy:  70.0


In [6]:
#random forest
from sklearn.ensemble import RandomForestClassifier
#n_estimators=400, random_state=42,max_depth=70
#model = RandomForestClassifier(n_estimators=400, random_state=42,max_depth=40,min_samples_split=5)
model = RandomForestClassifier(n_estimators=400, random_state=42,max_depth=40,min_samples_split=7)
model.fit(X_train, y_train) 
y_pred = model.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print('Accuracy: ', accuracy_score(y_test,y_pred))

[[167  16]
 [ 62  55]]
              precision    recall  f1-score   support

    NEGATIVE       0.73      0.91      0.81       183
    POSITIVE       0.77      0.47      0.59       117

    accuracy                           0.74       300
   macro avg       0.75      0.69      0.70       300
weighted avg       0.75      0.74      0.72       300

Accuracy:  0.74


In [6]:
#Save model
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)

In [8]:
#Load model
with open(pkl_filename, 'rb') as file:
    model = pickle.load(file)

In [9]:
#Text processing for gettin input
data = pd.read_csv('tweet.csv')
X, y= data['Tweet'], data['class_label']
documents = []
from nltk.stem import WordNetLemmatizer
stemmer = WordNetLemmatizer()
for sen in range(0, len(X)):
    document = re.sub(r'\W', ' ', str(X[sen]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.lower()
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    documents.append(document)
stw = stopwords.words('english')
addi = ("&amp","arc",'https','co','t','amp','s')
stw.append(addi)
vectorizer = CountVectorizer(max_features=1500, min_df=9, max_df=0.8, stop_words=stw)
X = vectorizer.fit_transform(documents).toarray()
def printPos():
    print("██████╗░░█████╗░░██████╗██╗████████╗██╗██╗░░░██╗███████╗\n"+
          "██╔══██╗██╔══██╗██╔════╝██║╚══██╔══╝██║██║░░░██║██╔════╝\n"+
          "██████╔╝██║░░██║╚█████╗░██║░░░██║░░░██║╚██╗░██╔╝█████╗░░\n"+
          "██╔═══╝░██║░░██║░╚═══██╗██║░░░██║░░░██║░╚████╔╝░██╔══╝░░\n"+
          "██║░░░░░╚█████╔╝██████╔╝██║░░░██║░░░██║░░╚██╔╝░░███████╗\n"+
          "╚═╝░░░░░░╚════╝░╚═════╝░╚═╝░░░╚═╝░░░╚═╝░░░╚═╝░░░╚══════╝")
def printNeg():
    print("███╗░░██╗███████╗░██████╗░░█████╗░████████╗██╗██╗░░░██╗███████╗\n"+
          "████╗░██║██╔════╝██╔════╝░██╔══██╗╚══██╔══╝██║██║░░░██║██╔════╝\n"+
          "██╔██╗██║█████╗░░██║░░██╗░███████║░░░██║░░░██║╚██╗░██╔╝█████╗░░\n"+
          "██║╚████║██╔══╝░░██║░░╚██╗██╔══██║░░░██║░░░██║░╚████╔╝░██╔══╝░░\n"+
          "██║░╚███║███████╗╚██████╔╝██║░░██║░░░██║░░░██║░░╚██╔╝░░███████╗\n"+
          "╚═╝░░╚══╝╚══════╝░╚═════╝░╚═╝░░╚═╝░░░╚═╝░░░╚═╝░░░╚═╝░░░╚══════╝")       


In [10]:
#Get input and predict the input
inputt = input("Enter your text")
print("Your Input :",inputt)
res = model.predict(vectorizer.transform([inputt]).toarray())
print("Your result\n")
if(res[0] == 'POSITIVE'):
    printPos()
else:
    printNeg()

Your Input : klkjok
Your result

███╗░░██╗███████╗░██████╗░░█████╗░████████╗██╗██╗░░░██╗███████╗
████╗░██║██╔════╝██╔════╝░██╔══██╗╚══██╔══╝██║██║░░░██║██╔════╝
██╔██╗██║█████╗░░██║░░██╗░███████║░░░██║░░░██║╚██╗░██╔╝█████╗░░
██║╚████║██╔══╝░░██║░░╚██╗██╔══██║░░░██║░░░██║░╚████╔╝░██╔══╝░░
██║░╚███║███████╗╚██████╔╝██║░░██║░░░██║░░░██║░░╚██╔╝░░███████╗
╚═╝░░╚══╝╚══════╝░╚═════╝░╚═╝░░╚═╝░░░╚═╝░░░╚═╝░░░╚═╝░░░╚══════╝
